## 回顾多层感知机
在这个例子中，通过实例化`nn.Sequential`来构建模型，层的执行顺序是作为参数传递的。  
**`nn.Sequential`定义了一种特殊的`Module`**，即在PyTorch中表示一个块的类，它维护了一个由`Module`组成的有序列表。  
注意，两个全连接层都是`Linear`类的实例，`Linear`类本身就是`Module`的子类。  
另外，通过`net(X)`调用模型来获得模型的输出，这实际上是`net.__call__(X)`的简写。

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256, 10))
X= torch.rand(2,20)
net(X)

tensor([[-0.0297, -0.0955, -0.1339, -0.1052, -0.0353, -0.1370, -0.0003, -0.1600,
          0.1311,  0.0293],
        [-0.0802, -0.1107, -0.1197, -0.1486, -0.2521, -0.1332, -0.0595, -0.2295,
          0.2202,  0.2215]], grad_fn=<AddmmBackward0>)

## 自定义块
下面的`MLP`类继承了表示块的类。实现只需要提供构造函数（Python中的`__init__`函数）和前向传播函数（`forward`函数）。  
1.定制的`__init__`函数通过`super().__init__()`调用父类的`__init__`函数。   
2.实例化两个全连接层，分别为`self.hidden`和`self.out`。  
3.前向传播函数，它以`X`作为输入，计算带有激活函数的隐藏表示，并输出其未规范化的输出值。  
在这个`MLP`实现中，两个层都是实例变量。

In [5]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.0327,  0.1768,  0.1115, -0.2158,  0.0729, -0.1511, -0.0554,  0.0229,
         -0.0271, -0.1190],
        [ 0.0175,  0.1743,  0.0817, -0.2650,  0.1565, -0.1770, -0.1179,  0.0101,
         -0.0270,  0.0449]], grad_fn=<AddmmBackward0>)

## 顺序块
构建自己的简化的`MySequential`，只需要定义两个关键函数：
1. 一种将块逐个追加到列表中的函数。
2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

为什么每个`Module`都有一个`_modules`属性，为什么使用它而不是自己定义一个Python列表？
`_modules`的主要优点是：在模块的参数初始化过程中，系统知道在`_modules`字典中查找需要初始化参数的子块。

In [7]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            # 这里，module是Module子类的一个实例。
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
            
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1928, -0.1240,  0.1268,  0.0776,  0.1725,  0.0035, -0.0161, -0.1311,
          0.1225,  0.0147],
        [-0.1538, -0.2169,  0.1455,  0.0109,  0.0809, -0.0580,  0.1365, -0.0690,
          0.0972,  0.1093]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码
到目前为止，我们网络中的所有操作都对网络的激活值及网络的参数起作用。  
然而，有时可能希望合并既不是上一层的结果也不是可更新参数的项，称之为*常数参数*（constant parameter）。  
例如，需要一个计算函数$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，$\mathbf{w}$是参数，$c$是某个在优化过程中没有更新的指定常量。  
因此实现了一个`FixedHiddenMLP`类，如下所示：

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在返回输出之前，模型运行了一个while循环，在$L_1$范数大于$1$的条件下，将输出向量除以$2$，直到它满足条件为止。最后，返回`X`中所有项的和。
注意，此操作可能不会常用于在任何实际任务中，只用于展示如何将任意代码集成到神经网络计算的流程中。

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(0.0484, grad_fn=<SumBackward0>)

## 混合搭配各种组合块

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1590, grad_fn=<SumBackward0>)